In [85]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [86]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [87]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


In [88]:
data = pd.read_csv("fashion-mnist_train.csv")
data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
data = data.fillna(0)
data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
X = data.drop("label", axis =1).to_numpy()
y = data["label"].to_numpy()

In [91]:
X = X/255.0
y = y/255.0

In [92]:
X_train, X_test,y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [93]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype = torch.float32)
    self.labels = torch.tensor(labels, dtype = torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]


In [94]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_train, y_test)

In [95]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size= 32, shuffle=False, pin_memory=True)

In [96]:
learning_rate = 0.00001
epochs = 50

In [97]:
class mynn(nn.Module):
  def __init__(self, num_dimension):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_dimension, 50),
        nn.ReLU(),
        nn.Linear(50, 25),
        nn.Sigmoid(),
        nn.Linear(25,10)
    )

  def forward(self, x):
    return self.model(x)


In [98]:
model = mynn(X_train.shape[1])
model = model.to(device)

In [99]:
optimizer  = optim.Adam(model.parameters(), lr = learning_rate)

In [100]:
criterion = nn.CrossEntropyLoss()

In [101]:
for epoch in  range(epochs):
  total_loss_epoch = 0
  for batch_features, batch_labels in train_dataloader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    outputs = model(batch_features)
    loss =criterion(outputs, batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss_epoch += loss.item()

  print(f"epoch: {epoch+1}, Loss: {total_loss_epoch/len(train_dataloader)}")



epoch: 1, Loss: 1.8563680052757263
epoch: 2, Loss: 1.340374752998352
epoch: 3, Loss: 1.0374336521625518
epoch: 4, Loss: 0.8480536444981893
epoch: 5, Loss: 0.7091984547773997
epoch: 6, Loss: 0.6014890303611755
epoch: 7, Loss: 0.5116273060242335
epoch: 8, Loss: 0.4307548925479253
epoch: 9, Loss: 0.35635021050771076
epoch: 10, Loss: 0.29560051878293353
epoch: 11, Loss: 0.2473345880707105
epoch: 12, Loss: 0.20738816497723261
epoch: 13, Loss: 0.17181832005580266
epoch: 14, Loss: 0.13838046344121296
epoch: 15, Loss: 0.1144932821492354
epoch: 16, Loss: 0.09518527747193972
epoch: 17, Loss: 0.07922433986266454
epoch: 18, Loss: 0.06595235098898411
epoch: 19, Loss: 0.05452415402730306
epoch: 20, Loss: 0.04313441612323125
epoch: 21, Loss: 0.03539683284362157
epoch: 22, Loss: 0.027949799858033655
epoch: 23, Loss: 0.022340368839601676
epoch: 24, Loss: 0.01830581639458736
epoch: 25, Loss: 0.015032826830943426
epoch: 26, Loss: 0.012348540668686231
epoch: 27, Loss: 0.010142872697363298
epoch: 28, Loss: